# Predicting Gang Violence

Using a data set from buzzfeed and local police departments, we will create a model based on features from the data to predict if a crime occured is gang realted or not.

## EDA

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
df = pd.read_csv('offenses-standardized.csv')
something = pd.read_csv('offenses-standardized.csv')

In [11]:
df.head()

,row_hash,source,agency_name,agency_ori,agency_incident_id,occurred_date,occurred_time,offense_category,top_ucr_classification,top_local_classification,...,victim_race,victim_ethnicity,victim_race_condensed,victim_sex,case_status,arrest_ind,clearance_status,clearance_group,clearance_date,arrest_date
0,MDBPD0000-C419771020-01-01,BALTIMORE-MAIN,BALTIMORE-PD,MDBPD0000,17V0351,2017-07-26,NaN,NON FATAL SHOOTING,NaN,NON FATAL SHOOTING,...,BLACK,NaN,BLACK/HISPANIC,MALE,OPEN,NaN,OPEN,Open & No Arrest-Unspecified,NaN,NaN
1,MDBPD0000-B63BECA76C-01-01,BALTIMORE-MAIN,BALTIMORE-PD,MDBPD0000,17H0198,2017-07-26,NaN,NON FATAL SHOOTING,NaN,NON FATAL SHOOTING,...,BLACK,NaN,BLACK/HISPANIC,FEMALE,OPEN,NaN,OPEN,Open & No Arrest-Unspecified,NaN,NaN
2,MDBPD0000-8A7079014F-01-01,BALTIMORE-MAIN,BALTIMORE-PD,MDBPD0000,17V0349,2017-07-25,NaN,NON FATAL SHOOTING,NaN,NON FATAL SHOOTING,...,BLACK,NaN,BLACK/HISPANIC,MALE,OPEN,NaN,OPEN,Open & No Arrest-Unspecified,NaN,NaN
3,MDBPD0000-3EF6B5EBEF-01-01,BALTIMORE-MAIN,BALTIMORE-PD,MDBPD0000,17V0348,2017-07-25,NaN,NON FATAL SHOOTING,NaN,NON FATAL SHOOTING,...,BLACK,NaN,BLACK/HISPANIC,MALE,OPEN,NaN,OPEN,Open & No Arrest-Unspecified,NaN,NaN
4,MDBPD0000-5995BFA411-01-01,BALTIMORE-MAIN,BALTIMORE-PD,MDBPD0000,17V0346,2017-07-24,NaN,NON FATAL SHOOTING,NaN,NON FATAL SHOOTING,...,BLACK,NaN,BLACK/HISPANIC,MALE,OPEN,NaN,OPEN,Open & No Arrest-Unspecified,NaN,NaN


In [56]:
df.top_local_classification.unique()

array(['NON FATAL SHOOTING', 'HOMICIDE SHOOTING', 'HOMICIDE BEATING', ...,
       'ASSAULT 2ND DEGREE INTENTIONAL INJURY TO PERSON RENDERING EMERGENCY CARE',
       'ASSAULT 2ND DEGREE INTENTIONAL INJURY TO AMBULANCE OPERATOR/NURSE/PARAMEDIC/MEDICAL DOCTOR WHILE PERFORMING WORK RELATED DUTY',
       'ASSAULT FIRST DEGREE- INTENTIONALLY DISFIGURE DESTROY AMPUTATE DISABLE'],
      dtype=object)

In [43]:
# I want to see all my columns, so I set my columns diplay options to max

pd.set_option('display.max_columns', None)

In [25]:
# We want crimes associated with gang activity, so we will be looking at the 'N' and 'Y' columns.

df.gang_activity_ind.value_counts(normalize = True)

N              0.584448
UNSPECIFIED    0.248238
Y              0.141642
UNKNOWN        0.020615
MISSING        0.003340
POS            0.000786
UNK            0.000511
UNCLEAR        0.000225
INV            0.000112
REL            0.000056
MAY            0.000011
NOT            0.000006
NP             0.000006
NO0            0.000006
Name: gang_activity_ind, dtype: float64

In [44]:
df.head()

,row_hash,source,agency_name,agency_ori,agency_incident_id,occurred_date,occurred_time,offense_category,top_ucr_classification,top_local_classification,nibrs_code,weapon,firearm_ind,offense_group,domestic_violence_ind,family_violence_ind,gang_activity_ind,relationship_to_offender,circumstance,inside_outside,victim_age,victim_race,victim_ethnicity,victim_race_condensed,victim_sex,case_status,arrest_ind,clearance_status,clearance_group,clearance_date,arrest_date
0,MDBPD0000-C419771020-01-01,BALTIMORE-MAIN,BALTIMORE-PD,MDBPD0000,17V0351,2017-07-26,NaN,NON FATAL SHOOTING,NaN,NON FATAL SHOOTING,NaN,FIREARM,Y,Non-Fatal Shooting,NaN,NaN,NaN,NaN,NaN,NaN,22,BLACK,NaN,BLACK/HISPANIC,MALE,OPEN,NaN,OPEN,Open & No Arrest-Unspecified,NaN,NaN
1,MDBPD0000-B63BECA76C-01-01,BALTIMORE-MAIN,BALTIMORE-PD,MDBPD0000,17H0198,2017-07-26,NaN,NON FATAL SHOOTING,NaN,NON FATAL SHOOTING,NaN,FIREARM,Y,Non-Fatal Shooting,NaN,NaN,NaN,NaN,NaN,NaN,53,BLACK,NaN,BLACK/HISPANIC,FEMALE,OPEN,NaN,OPEN,Open & No Arrest-Unspecified,NaN,NaN
2,MDBPD0000-8A7079014F-01-01,BALTIMORE-MAIN,BALTIMORE-PD,MDBPD0000,17V0349,2017-07-25,NaN,NON FATAL SHOOTING,NaN,NON FATAL SHOOTING,NaN,FIREARM,Y,Non-Fatal Shooting,NaN,NaN,NaN,NaN,NaN,NaN,19,BLACK,NaN,BLACK/HISPANIC,MALE,OPEN,NaN,OPEN,Open & No Arrest-Unspecified,NaN,NaN
3,MDBPD0000-3EF6B5EBEF-01-01,BALTIMORE-MAIN,BALTIMORE-PD,MDBPD0000,17V0348,2017-07-25,NaN,NON FATAL SHOOTING,NaN,NON FATAL SHOOTING,NaN,FIREARM,Y,Non-Fatal Shooting,NaN,NaN,NaN,NaN,NaN,NaN,29,BLACK,NaN,BLACK/HISPANIC,MALE,OPEN,NaN,OPEN,Open & No Arrest-Unspecified,NaN,NaN
4,MDBPD0000-5995BFA411-01-01,BALTIMORE-MAIN,BALTIMORE-PD,MDBPD0000,17V0346,2017-07-24,NaN,NON FATAL SHOOTING,NaN,NON FATAL SHOOTING,NaN,FIREARM,Y,Non-Fatal Shooting,NaN,NaN,NaN,NaN,NaN,NaN,22,BLACK,NaN,BLACK/HISPANIC,MALE,OPEN,NaN,OPEN,Open & No Arrest-Unspecified,NaN,NaN


In [68]:
df.family_violence_ind.value_counts(normalize = True)

N          0.915438
Y          0.080121
MISSING    0.004330
UNCLEAR    0.000111
Name: family_violence_ind, dtype: float64

In [31]:
df.isnull().sum()

row_hash                         0
source                           0
agency_name                      0
agency_ori                       0
agency_incident_id               0
occurred_date                   86
occurred_time                18564
offense_category            316594
top_ucr_classification      108303
top_local_classification     44977
nibrs_code                  710134
weapon                       51689
firearm_ind                      0
offense_group                    0
domestic_violence_ind       278195
family_violence_ind         702626
gang_activity_ind           569490
relationship_to_offender    582469
circumstance                729749
inside_outside              744816
victim_age                  469869
victim_race                 459988
victim_ethnicity            546069
victim_race_condensed            0
victim_sex                  460048
case_status                 419985
arrest_ind                   59759
clearance_status            390403
clearance_group     

In [22]:
yes = df.loc[df['gang_activity_ind'] == 'Y']

In [24]:
yes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25236 entries, 274808 to 502656
Data columns (total 31 columns):
row_hash                    25236 non-null object
source                      25236 non-null object
agency_name                 25236 non-null object
agency_ori                  25236 non-null object
agency_incident_id          25236 non-null object
occurred_date               25236 non-null object
occurred_time               24824 non-null object
offense_category            25112 non-null object
top_ucr_classification      21838 non-null object
top_local_classification    24823 non-null object
nibrs_code                  287 non-null object
weapon                      24358 non-null object
firearm_ind                 25236 non-null object
offense_group               25236 non-null object
domestic_violence_ind       3922 non-null object
family_violence_ind         996 non-null object
gang_activity_ind           25236 non-null object
relationship_to_offender    3930 non-nul

In [66]:
df.head()

,row_hash,source,agency_name,agency_ori,agency_incident_id,occurred_date,occurred_time,offense_category,top_ucr_classification,top_local_classification,nibrs_code,weapon,firearm_ind,offense_group,domestic_violence_ind,family_violence_ind,gang_activity_ind,relationship_to_offender,circumstance,inside_outside,victim_age,victim_race,victim_ethnicity,victim_race_condensed,victim_sex,case_status,arrest_ind,clearance_status,clearance_group,clearance_date,arrest_date
0,MDBPD0000-C419771020-01-01,BALTIMORE-MAIN,BALTIMORE-PD,MDBPD0000,17V0351,2017-07-26,NaN,NON FATAL SHOOTING,NaN,NON FATAL SHOOTING,NaN,FIREARM,Y,Non-Fatal Shooting,NaN,NaN,NaN,NaN,NaN,NaN,22,BLACK,NaN,BLACK/HISPANIC,MALE,OPEN,NaN,OPEN,Open & No Arrest-Unspecified,NaN,NaN
1,MDBPD0000-B63BECA76C-01-01,BALTIMORE-MAIN,BALTIMORE-PD,MDBPD0000,17H0198,2017-07-26,NaN,NON FATAL SHOOTING,NaN,NON FATAL SHOOTING,NaN,FIREARM,Y,Non-Fatal Shooting,NaN,NaN,NaN,NaN,NaN,NaN,53,BLACK,NaN,BLACK/HISPANIC,FEMALE,OPEN,NaN,OPEN,Open & No Arrest-Unspecified,NaN,NaN
2,MDBPD0000-8A7079014F-01-01,BALTIMORE-MAIN,BALTIMORE-PD,MDBPD0000,17V0349,2017-07-25,NaN,NON FATAL SHOOTING,NaN,NON FATAL SHOOTING,NaN,FIREARM,Y,Non-Fatal Shooting,NaN,NaN,NaN,NaN,NaN,NaN,19,BLACK,NaN,BLACK/HISPANIC,MALE,OPEN,NaN,OPEN,Open & No Arrest-Unspecified,NaN,NaN
3,MDBPD0000-3EF6B5EBEF-01-01,BALTIMORE-MAIN,BALTIMORE-PD,MDBPD0000,17V0348,2017-07-25,NaN,NON FATAL SHOOTING,NaN,NON FATAL SHOOTING,NaN,FIREARM,Y,Non-Fatal Shooting,NaN,NaN,NaN,NaN,NaN,NaN,29,BLACK,NaN,BLACK/HISPANIC,MALE,OPEN,NaN,OPEN,Open & No Arrest-Unspecified,NaN,NaN
4,MDBPD0000-5995BFA411-01-01,BALTIMORE-MAIN,BALTIMORE-PD,MDBPD0000,17V0346,2017-07-24,NaN,NON FATAL SHOOTING,NaN,NON FATAL SHOOTING,NaN,FIREARM,Y,Non-Fatal Shooting,NaN,NaN,NaN,NaN,NaN,NaN,22,BLACK,NaN,BLACK/HISPANIC,MALE,OPEN,NaN,OPEN,Open & No Arrest-Unspecified,NaN,NaN


In [76]:
df2 = df.drop(columns = ['row_hash','arrest_date','clearance_date','clearance_group','clearance_status','case_status',
                   'victim_race_condensed','victim_ethnicity','circumstance','relationship_to_offender',
                  'weapon','nibrs_code','top_local_classification','top_ucr_classification',
                  'agency_name','agency_ori','agency_incident_id', 'occurred_time'])
df2

,source,occurred_date,offense_category,firearm_ind,offense_group,domestic_violence_ind,family_violence_ind,gang_activity_ind,inside_outside,victim_age,victim_race,victim_sex,arrest_ind
0,BALTIMORE-MAIN,2017-07-26,NON FATAL SHOOTING,Y,Non-Fatal Shooting,NaN,NaN,NaN,NaN,22,BLACK,MALE,NaN
1,BALTIMORE-MAIN,2017-07-26,NON FATAL SHOOTING,Y,Non-Fatal Shooting,NaN,NaN,NaN,NaN,53,BLACK,FEMALE,NaN
2,BALTIMORE-MAIN,2017-07-25,NON FATAL SHOOTING,Y,Non-Fatal Shooting,NaN,NaN,NaN,NaN,19,BLACK,MALE,NaN
3,BALTIMORE-MAIN,2017-07-25,NON FATAL SHOOTING,Y,Non-Fatal Shooting,NaN,NaN,NaN,NaN,29,BLACK,MALE,NaN
4,BALTIMORE-MAIN,2017-07-24,NON FATAL SHOOTING,Y,Non-Fatal Shooting,NaN,NaN,NaN,NaN,22,BLACK,MALE,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
747653,WILMINGTON-MAIN,2018-03-18,AGGRAVATED ASSAULT,Y,Agg Assault - Gun,N,N,NaN,NaN,NaN,NaN,NaN,N
747654,WILMINGTON-MAIN,2018-03-18,AGGRAVATED ASSAULT,Y,Agg Assault - Gun,N,N,NaN,NaN,NaN,NaN,NaN,N
747655,WILMINGTON-MAIN,2018-03-18,AGGRAVATED ASSAULT,N,Agg Assault - Other Weapon,N,N,NaN,NaN,NaN,NaN,NaN,N
747656,WILMINGTON-MAIN,2018-03-20,AGGRAVATED ASSAULT,N,Agg Assault - Other Weapon,N,N,NaN,NaN,NaN,NaN,NaN,N


In [72]:
df2.offense_category.value_counts()

AGGRAVATED ASSAULT                               160633
AGG ASSAULT                                      140043
AGG ASSAULT - OTHER WEAPON                        64534
HOMICIDE                                          20360
AGG ASSAULT - FIREARM - DISCHARGE UNSPECIFIED      8999
AGG ASSAULT - SHOOTING - NON-FATAL                 7426
MURDER                                             5609
AGG ASSAULT - SHOOTING - HIT UNSPECIFIED           5589
AGG ASSAULT - UNSPECIFIED WEAPON                   4030
NON-FATAL SHOOTING                                 3755
NON FATAL SHOOTING                                 3621
AGG ASSAULT - SHOOTING - NO HIT                    3076
HOMICIDE - CRIMINAL - SHOOTING - FATAL             1730
NON-FATAL-SHOOTING                                  908
HOMICIDE - CRIMINAL - OTHER WEAPON                  608
MURDER AND NON-NEGLIGENT MANSLAUGHTER               138
HOMICIDE - CRIMINAL - UNSPECIFIED WEAPON              5
Name: offense_category, dtype: int64

In [91]:
df2.offense_category = df2.offense_category.replace(to_replace ='AGGRAVATED ASSAULT', value =  'AGGRAVATED_ASSAULT')

In [87]:
df2.rename(columns = {'AGGRAVATED ASSAULT': 'AGGRAVATED_ASSAULT'}, inplace = True)

In [167]:
df2.offense_category.replace(to_replace= 'AGG ASSAULT', value = 'AGGRAVATED_ASSAULT', inplace = True)

In [170]:
df2.offense_category.value_counts()

AGGRAVATED_ASSAULT                               300676
AGG ASSAULT - OTHER WEAPON                        64534
HOMICIDE                                          20360
AGG ASSAULT - FIREARM - DISCHARGE UNSPECIFIED      8999
AGG ASSAULT - SHOOTING - NON-FATAL                 7426
MURDER                                             5609
AGG ASSAULT - SHOOTING - HIT UNSPECIFIED           5589
AGG ASSAULT - UNSPECIFIED WEAPON                   4030
NON-FATAL SHOOTING                                 3755
NON FATAL SHOOTING                                 3621
AGG ASSAULT - SHOOTING - NO HIT                    3076
HOMICIDE - CRIMINAL - SHOOTING - FATAL             1730
NON-FATAL-SHOOTING                                  908
HOMICIDE - CRIMINAL - OTHER WEAPON                  608
MURDER AND NON-NEGLIGENT MANSLAUGHTER               138
HOMICIDE - CRIMINAL - UNSPECIFIED WEAPON              5
Name: offense_category, dtype: int64

In [173]:
df2.offense_category.replace(to_replace= ['AGG ASSAULT - OTHER WEAPON','AGG ASSAULT - FIREARM - DISCHARGE UNSPECIFIED'], value = 'AGGRAVATED_ASSAULT', inplace = True)


In [175]:
df2.offense_category.value_counts()

AGGRAVATED_ASSAULT                          374209
HOMICIDE                                     20360
AGG ASSAULT - SHOOTING - NON-FATAL            7426
MURDER                                        5609
AGG ASSAULT - SHOOTING - HIT UNSPECIFIED      5589
AGG ASSAULT - UNSPECIFIED WEAPON              4030
NON-FATAL SHOOTING                            3755
NON FATAL SHOOTING                            3621
AGG ASSAULT - SHOOTING - NO HIT               3076
HOMICIDE - CRIMINAL - SHOOTING - FATAL        1730
NON-FATAL-SHOOTING                             908
HOMICIDE - CRIMINAL - OTHER WEAPON             608
MURDER AND NON-NEGLIGENT MANSLAUGHTER          138
HOMICIDE - CRIMINAL - UNSPECIFIED WEAPON         5
Name: offense_category, dtype: int64

In [176]:
df2.offense_category.replace(to_replace= ['AGG ASSAULT - SHOOTING - NON-FATAL','AGG ASSAULT - SHOOTING - HIT UNSPECIFIED','AGG ASSAULT - UNSPECIFIED WEAPON', 'AGG ASSAULT - SHOOTING - NO HIT'], value = 'AGGRAVATED_ASSAULT', inplace = True)


In [177]:
df2.offense_category.value_counts()

AGGRAVATED_ASSAULT                          394330
HOMICIDE                                     20360
MURDER                                        5609
NON-FATAL SHOOTING                            3755
NON FATAL SHOOTING                            3621
HOMICIDE - CRIMINAL - SHOOTING - FATAL        1730
NON-FATAL-SHOOTING                             908
HOMICIDE - CRIMINAL - OTHER WEAPON             608
MURDER AND NON-NEGLIGENT MANSLAUGHTER          138
HOMICIDE - CRIMINAL - UNSPECIFIED WEAPON         5
Name: offense_category, dtype: int64

In [119]:
df2.offense_category.dropna(inplace = True)
df2.offense_category.unique()

array(['NON FATAL SHOOTING', 'HOMICIDE', 'NON-FATAL SHOOTING', 'MURDER',
       'AGGRAVATED_ASSAULT', 'MURDER AND NON-NEGLIGENT MANSLAUGHTER',
       'AGG ASSAULT - SHOOTING - NON-FATAL', 'AGG ASSAULT - OTHER WEAPON',
       'AGG ASSAULT - UNSPECIFIED WEAPON',
       'AGG ASSAULT - SHOOTING - HIT UNSPECIFIED',
       'HOMICIDE - CRIMINAL - SHOOTING - FATAL',
       'HOMICIDE - CRIMINAL - OTHER WEAPON',
       'AGG ASSAULT - FIREARM - DISCHARGE UNSPECIFIED',
       'HOMICIDE - CRIMINAL - UNSPECIFIED WEAPON',
       'AGG ASSAULT - SHOOTING - NO HIT', 'AGG ASSAULT',
       'NON-FATAL-SHOOTING'], dtype=object)

In [137]:
df2.offense_category.value_counts()

AGGRAVATED_ASSAULT                               160633
AGG ASSAULT                                      140043
AGG ASSAULT - OTHER WEAPON                        64534
HOMICIDE                                          20360
AGG ASSAULT - FIREARM - DISCHARGE UNSPECIFIED      8999
AGG ASSAULT - SHOOTING - NON-FATAL                 7426
MURDER                                             5609
AGG ASSAULT - SHOOTING - HIT UNSPECIFIED           5589
AGG ASSAULT - UNSPECIFIED WEAPON                   4030
NON-FATAL SHOOTING                                 3755
NON FATAL SHOOTING                                 3621
AGG ASSAULT - SHOOTING - NO HIT                    3076
HOMICIDE - CRIMINAL - SHOOTING - FATAL             1730
NON-FATAL-SHOOTING                                  908
HOMICIDE - CRIMINAL - OTHER WEAPON                  608
MURDER AND NON-NEGLIGENT MANSLAUGHTER               138
HOMICIDE - CRIMINAL - UNSPECIFIED WEAPON              5
Name: offense_category, dtype: int64

In [160]:
df2.offense_category.unique()[4] = df2.offense_category.unique()[6] + df2.offense_category.unique()[4] +df2.offense_category.unique()[7] + df2.offense_category.unique()[8] + df2.offense_category.unique()[9] + df2.offense_category.unique()[13] + df2.offense_category.unique()[15] + df2.offense_category.unique()[16]

In [161]:
df2.offense_category.unique()[4]

'AGGRAVATED_ASSAULT'

In [162]:
df2.offense_category.unique()

array(['NON FATAL SHOOTING', 'HOMICIDE', 'NON-FATAL SHOOTING', 'MURDER',
       'AGGRAVATED_ASSAULT', 'MURDER AND NON-NEGLIGENT MANSLAUGHTER',
       'AGG ASSAULT - SHOOTING - NON-FATAL', 'AGG ASSAULT - OTHER WEAPON',
       'AGG ASSAULT - UNSPECIFIED WEAPON',
       'AGG ASSAULT - SHOOTING - HIT UNSPECIFIED',
       'HOMICIDE - CRIMINAL - SHOOTING - FATAL',
       'HOMICIDE - CRIMINAL - OTHER WEAPON',
       'AGG ASSAULT - FIREARM - DISCHARGE UNSPECIFIED',
       'HOMICIDE - CRIMINAL - UNSPECIFIED WEAPON',
       'AGG ASSAULT - SHOOTING - NO HIT', 'AGG ASSAULT',
       'NON-FATAL-SHOOTING'], dtype=object)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747658 entries, 0 to 747657
Data columns (total 31 columns):
row_hash                    747658 non-null object
source                      747658 non-null object
agency_name                 747658 non-null object
agency_ori                  747658 non-null object
agency_incident_id          747658 non-null object
occurred_date               747572 non-null object
occurred_time               729094 non-null object
offense_category            431064 non-null object
top_ucr_classification      639355 non-null object
top_local_classification    702681 non-null object
nibrs_code                  37524 non-null object
weapon                      695969 non-null object
firearm_ind                 747658 non-null object
offense_group               747658 non-null object
domestic_violence_ind       469463 non-null object
family_violence_ind         45032 non-null object
gang_activity_ind           178168 non-null object
relationship_to_offender